In [12]:
# Solution 4.3
import numpy as np
# 已知条件：中性面应变和曲率
epsilon_0 = np.array([[2E-6], [3e-6], [4e-6]])
kappa_0 = np.array([[1.2e-4], [1.5e-4], [2.6e-4]])
# 层合板厚度为 0.015
# 计算层合板上表面的全局应变：
epsilon_top = epsilon_0 + (-0.0075)* kappa_0
print("层合板上表面的全局应变：\n", epsilon_top)
# 计算层合板中性面的全局应变：
epsilon_mid = epsilon_0 + (0.0)* kappa_0
print("层合板中性面的全局应变：\n", epsilon_mid)
# 计算层合板下表面的全局应变：
epsilon_bottom = epsilon_0 + (0.0075)* kappa_0
print("层合板下表面的全局应变：\n", epsilon_bottom)

层合板上表面的全局应变：
 [[1.100e-06]
 [1.875e-06]
 [2.050e-06]]
层合板中性面的全局应变：
 [[2.e-06]
 [3.e-06]
 [4.e-06]]
层合板下表面的全局应变：
 [[2.900e-06]
 [4.125e-06]
 [5.950e-06]]


In [13]:
# 定义函数由单层板的工程常数计算单层板折减刚度矩阵Q
def transform_engineeringConstants(E1, E2, v12, G12):
    # 根据工程常数计算柔度矩阵的系数
    S11 = 1/E1;
    S12 = -v12/E1;
    S22 = 1/E2;
    S66 = 1/G12;
    # 初始化柔度矩阵
    S = np.zeros((3,3))
    # 将柔度矩阵的非零元素依次赋值；注意Python的编码从0开始。
    S[0][0] = S11;
    S[0][1] = S12;
    S[1][0] = S[0][1];
    S[1][1] = S22;
    S[2][2] = S66;
    # 矩阵运算求解折减刚度矩阵
    Q = np.linalg.inv(S)
    return Q

In [14]:
# 定义函数由单层板折减刚度矩阵Q和铺层角度，计算单层板在全局坐标系下的折减刚度矩阵Q_bar
def transform_Q(Q, theta):
    # 计算转换矩阵 T
    c = np.cos(theta*np.pi/180.0)
    s = np.sin(theta*np.pi/180.0)
    T = np.zeros((3,3))
    T[0][0] = c**2;
    T[0][1] = s**2;
    T[0][2] = 2.0*c*s;
    T[1][0] = s**2;
    T[1][1] = c**2;
    T[1][2] = -2.0*c*s;
    T[2][0] = -s*c;
    T[2][1] = s*c;
    T[2][2] = c**2-s**2;
    # 定义R矩阵
    R = np.zeros((3,3))
    R[0][0] = 1;
    R[1][1] = 1;
    R[2][2] = 2;
    # 计算全局折减刚度矩阵
    Q_bar = np.linalg.multi_dot([np.linalg.inv(T), Q, R, T, np.linalg.inv(R)])
    return Q_bar

In [15]:
def height(thickness):
    # thickness = np.array([5, 5, 5])
    ply_num = len(thickness)
    h_total = np.sum(thickness[0:ply_num]) # 层合板总厚度
    height = np.zeros(ply_num+1)
    #Create array storing the heights of k plies about the midplane
    for k in range(0, ply_num+1):
        height[k]= - h_total/2.0 + np.sum(thickness[0:k])
    return height

In [16]:
def ABD_matrices(Q, statck, thickness):
    A = np.zeros([3, 3])
    B = np.zeros([3, 3])
    D = np.zeros([3, 3])
    h = height(thickness)
    for i in range(3):
        for j in range(3):
            for k in range(len(h)-1):
                Q_bar = transform_Q(Q, statck[k])
                ply_kA = h[k+1] - h[k]
                ply_kB = (h[k+1]**2 - h[k]**2)/2
                ply_kD = (h[k+1]**3 - h[k]**3)/3
                A[i,j] = A[i,j] + ply_kA * Q_bar[i,j]
                B[i,j] = B[i,j] + ply_kB * Q_bar[i,j]
                D[i,j] = D[i,j] + ply_kD * Q_bar[i,j]
    return (A, B, D)

In [17]:
# Solution 4.8
Q = transform_engineeringConstants(5.6*1.0E6, 1.2*1.0E6, 0.26, 0.6*1.0E6) 
thickness = np.array([0.005, 0.005, 0.005])
statck = np.array([0, 60, -60])
# 注意 ABD_matrices 中包含A, B, D共三个矩阵，每个矩阵都是3X3的
A = ABD_matrices(Q, statck, thickness)[0]
B = ABD_matrices(Q, statck, thickness)[1]
D = ABD_matrices(Q, statck, thickness)[2]
print("A = ", A) 
print("B = ", B)
print("D = ", D)

A =  [[4.44994202e+04 1.19989998e+04 4.54747351e-13]
 [1.19989998e+04 4.44994202e+04 1.81898940e-12]
 [4.54747351e-13 9.09494702e-13 1.62502102e+04]]
B =  [[-101.83816282   18.12552547  -13.70097984]
 [  18.12552547   65.58711188  -34.63053386]
 [ -13.70097984  -34.63053386   18.12552547]]
D =  [[ 1.0040944   0.19477204 -0.0685049 ]
 [ 0.19477204  0.72505228 -0.17315267]
 [-0.0685049  -0.17315267  0.27448223]]


In [24]:
# 层合板质量计算
# 层合板体积v_c
v_c = 5.*7.*0.015;
# 纤维密度rho_f，基体密度rho_m # 注意需要将kg/m3 转换为 lbm/in3
rho_f = 2.5*1000*3.6127e-5; rho_m = 1.2*1000*3.6127e-5;
# 查表格2.2得到纤维体积分数为0.45，计算层合板密度rho_c
rho_c = rho_f * 0.45 + rho_m * (1.0-0.45)
# 层合板质量m
m = rho_c * v_c
print("层合板质量为：", m) 

层合板质量为： 0.033855514875000006


### Solution 4.9
$A  = \begin{bmatrix}
\frac{E}{1-v^2} & \frac{Ev}{1-v^2} & 0 \\ 
\frac{Ev}{1-v^2} & \frac{E}{1-v^2} & 0 \\ 
 0 & 0 & \frac{E}{2(1+v)} 
\end{bmatrix}t$

$B = 0$

$D  = \frac{1}{12} \begin{bmatrix}
\frac{E}{1-v^2} & \frac{Ev}{1-v^2} & 0 \\ 
\frac{Ev}{1-v^2} & \frac{E}{1-v^2} & 0 \\ 
 0 & 0 & \frac{E}{2(1+v)} 
\end{bmatrix}t^3$



In [ ]:
# Solution 4.14
Q = transform_engineeringConstants(26.25*1.0E6, 1.49*1.0E6, 0.28, 1.04*1.0E6) 
thickness = np.array([0.005, 0.005, 0.005])
statck = np.array([0, 60, -60])
# 注意 ABD_matrices 中包含A, B, D共三个矩阵，每个矩阵都是3X3的
A = ABD_matrices(Q, statck, thickness)[0]
B = ABD_matrices(Q, statck, thickness)[1]
D = ABD_matrices(Q, statck, thickness)[2]
print("A = ", A) 
print("B = ", B)
print("D = ", D)